In [1]:
import os
import requests
from dotenv import load_dotenv, find_dotenv
import pandas as pd

pd.set_option('display.max_columns', None)

# Local module
from pathlib import Path
import sys

# assumes notebook is in xapi-ex1/notebooks
project_root = Path.cwd().parent  
sys.path.append(str(project_root / 'src'))

from users import (
    get_recent_followings_cached, 
    get_recent_followers_cached,
    load_follow_cache,
    save_follow_cache,
    search_tweets_advanced,
    get_user_tweets, 
    get_user_tweets_cached,
    save_tweet_cache,
    load_tweet_cache,
)
from schema.tweets import (
    collapse_dicts, 
    collapse_dataframe, 
    TRUNCATED_TWEET_FIELDS
)

# Load env
load_dotenv(find_dotenv())
API_KEY = os.getenv('twitter_apiio_key')
if not API_KEY:
    raise RuntimeError('twitter_apiio_key not set in environment')

In [2]:
# Params
USERNAME = 'nelvOfficial' 
FOLLOWING_LIMIT = 200  # multiples of 20-200 recommended
FOLLOWERS_LIMIT = 600

In [3]:
# Get followings
followings = get_recent_followings_cached(USERNAME, limit=FOLLOWING_LIMIT, api_key=API_KEY)

✅ Cache hit: 200 followings


In [4]:
# Normalize to DataFrame
df_followings = pd.json_normalize(followings.get('followings', []))
print(f"Rows: {len(df_followings)}")

df_followings.head(2)

Rows: 200


,id,name,screen_name,userName,location,url,description,email,protected,verified,followers_count,following_count,friends_count,favourites_count,statuses_count,media_tweets_count,created_at,profile_banner_url,profile_image_url_https,can_dm
0,7699362,Nathan Snell,NathanSnell,NathanSnell,,https://t.co/wSNk44He1q,Cofounder & CEO @RaleonHQ - fully automate you...,None,False,False,2223,1151,1151,1663,2146,349,Wed Jul 25 01:04:00 +0000 2007,https://pbs.twimg.com/profile_banners/7699362/...,https://pbs.twimg.com/profile_images/159342471...,False
1,1629336979,Elizabeth Holmes,ElizabethHolmes,ElizabethHolmes,"Bryan, TX",None,Building a better world for my two children.\n...,None,False,False,47598,56,56,2652,719,45,Mon Jul 29 02:51:00 +0000 2013,https://pbs.twimg.com/profile_banners/16293369...,https://pbs.twimg.com/profile_images/593983848...,False


In [5]:
df_followings.iloc[0]

id                                                                   7699362
name                                                            Nathan Snell
screen_name                                                      NathanSnell
userName                                                         NathanSnell
location                                                                    
url                                                  https://t.co/wSNk44He1q
description                Cofounder & CEO @RaleonHQ - fully automate you...
email                                                                   None
protected                                                              False
verified                                                               False
followers_count                                                         2223
following_count                                                         1151
friends_count                                                           1151

## Get tweets for all followers

In [15]:
TWEET_LIMIT = 20
start_date = None
end_date = None
min_faves = 1
INCLUDE_REPLIES = False

In [16]:
resp = get_user_tweets_cached(
    api_key=API_KEY,
    username=USERNAME,
    limit=TWEET_LIMIT,
    start_date=start_date,
    end_date=end_date,
    min_faves=min_faves,
    include_replies=INCLUDE_REPLIES,
)
tweets = resp["tweets"]

🔄 C-Miss: Fetching with Q: `from:nelvOfficial min_faves:1 -is:reply`
💾 Cached 20 tweets


In [17]:
resp

{'has_next_page': True,
 'next_cursor': 'DAADDAABCgABG0a9htYbgCkKAAIbRCi3y9sgjgAIAAIAAAACCAADAAAAAAgABAAAAAAKAAUbRsw6JUAnEAoABhtGzDolP9jwAAA',
 'status': 'success',
 'message': 'Fetched 20 tweets for query: `from:nelvOfficial min_faves:1 -is:reply`',
 'tweets': [{'type': 'tweet',
   'id': '1965466674209194025',
   'url': 'https://x.com/nelvOfficial/status/1965466674209194025',
   'twitterUrl': 'https://twitter.com/nelvOfficial/status/1965466674209194025',
   'text': '@MMatt14 neuroplasticiity',
   'source': 'Twitter for iPhone',
   'retweetCount': 0,
   'replyCount': 0,
   'likeCount': 1,
   'quoteCount': 0,
   'viewCount': 3,
   'createdAt': 'Tue Sep 09 17:25:46 +0000 2025',
   'lang': 'und',
   'bookmarkCount': 0,
   'isReply': True,
   'inReplyToId': '1965464985330745521',
   'conversationId': '1965464985330745521',
   'displayTextRange': [9, 25],
   'inReplyToUserId': '485793018',
   'inReplyToUsername': 'MMatt14',
   'author': {'type': 'user',
    'userName': 'nelvOfficial',
    '